In [1]:
import torch
import json
import os
from pathlib import Path
from typing import Optional
from dataclasses import dataclass, field

import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq,
)
from datasets import load_dataset, Dataset
from peft import (
    get_peft_model,
    LoraConfig,
    TaskType,
    prepare_model_for_kbit_training,
)
import bitsandbytes as bnb

Exception in thread Thread-4 (_readerthread):
Traceback (most recent call last):
  File "c:\Users\helen\anaconda3\envs\LLMenv\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\helen\anaconda3\envs\LLMenv\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\helen\anaconda3\envs\LLMenv\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\helen\anaconda3\envs\LLMenv\lib\subprocess.py", line 1515, in _readerthread
    buffer.append(fh.read())
  File "c:\Users\helen\anaconda3\envs\LLMenv\lib\codecs.py", line 322, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc0 in position 6: invalid start byte


In [2]:
# ========== GPU 메모리 확인 ==========
print("GPU 정보:")
print(f"GPU Name: {torch.cuda.get_device_name(0)}")
print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

GPU 정보:
GPU Name: NVIDIA GeForce RTX 4060
GPU Memory: 8.00 GB


In [3]:
# ========== 설정 ==========
@dataclass
class Config:
    # 모델 설정
    model_name: str = "Qwen/Qwen2.5-1.5B-Instruct"  
    # model_name: str = "meta-llama/Llama-2-7b-hf"  # 7B 모델 (16GB 이상)
    
    # 데이터 경로
    train_data_path: str = "HTP_data.jsonl"
    
    # 출력 경로
    output_dir: str = "./htp_lora_model"
    
    # 배치 크기 (RTX 4060 8GB용)
    per_device_train_batch_size: int = 2
    per_device_eval_batch_size: int = 4
    gradient_accumulation_steps: int = 4
    
    # 학습 설정
    num_train_epochs: int = 10
    learning_rate: float = 2e-4
    warmup_ratio: float = 0.03
    
    # 텍스트 길이
    max_seq_length: int = 512
    
    # LoRA 설정
    lora_r: int = 8
    lora_alpha: int = 32
    lora_dropout: float = 0.05
    
    # 8-bit 양자화 (메모리 절감)
    use_8bit: bool = True
    
    # 기타
    seed: int = 42
    fp16: bool = True  # Mixed precision training


# ========== 데이터 로딩 ==========
def load_jsonl_data(file_path: str) -> list:
    """JSONL 파일 읽기"""
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                data.append(json.loads(line))
    return data


def prepare_dataset(data: list, tokenizer, config: Config) -> Dataset:
    """데이터셋 준비"""
    
    def formatting_func(examples):
        # input과 output을 합쳐서 학습 데이터 생성
        texts = []
        for inp, out in zip(examples['input'], examples['output']):
            text = f"HTP 해석 입력: {inp}\n\nHTP 해석 출력: {out}"
            texts.append(text)
        
        # 토큰화
        tokenized = tokenizer(
            texts,
            max_length=config.max_seq_length,
            padding="max_length",
            truncation=True,
            return_tensors=None,
        )
        
        # labels 설정 (input_ids와 동일하게)
        tokenized["labels"] = tokenized["input_ids"].copy()
        
        return tokenized
    
    # Dataset으로 변환
    dataset = Dataset.from_dict({
        'input': [item['input'] for item in data],
        'output': [item['output'] for item in data],
    })
    
    # 처리
    processed_dataset = dataset.map(
        formatting_func,
        batched=True,
        batch_size=100,
        remove_columns=['input', 'output'],
        desc="포매팅 중...",
    )
    
    return processed_dataset


# ========== 모델 로딩 및 LoRA 설정 ==========
def setup_model_and_tokenizer(config: Config):
    """모델과 토크나이저 설정"""
    
    print(f"모델 로딩: {config.model_name}")
    
    # 토크나이저 로딩
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    # 모델 로딩 (8-bit 양자화)
    if config.use_8bit:
        model = AutoModelForCausalLM.from_pretrained(
            config.model_name,
            load_in_8bit=True,
            torch_dtype=torch.float16,
            device_map="auto",
        )
        
        # 8-bit 양자화 모델 준비
        model = prepare_model_for_kbit_training(
            model,
            use_gradient_checkpointing=True,
        )
    else:
        model = AutoModelForCausalLM.from_pretrained(
            config.model_name,
            torch_dtype=torch.float16,
            device_map="auto",
        )
    
    # LoRA 설정
    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        r=config.lora_r,
        lora_alpha=config.lora_alpha,
        lora_dropout=config.lora_dropout,
        bias="none",
        target_modules=["q_proj", "v_proj"],  # 모델에 따라 수정 필요
    )
    
    # LoRA 적용
    model = get_peft_model(model, peft_config)
    
    # 학습 가능한 파라미터 확인
    model.print_trainable_parameters()
    
    return model, tokenizer

In [4]:
# ========== 메인 학습 함수 ==========
config = Config()

# 시드 설정
transformers.set_seed(config.seed)

# 출력 디렉토리 생성
Path(config.output_dir).mkdir(parents=True, exist_ok=True)

# 1. 데이터 로드
print("데이터 로딩 중...")
raw_data = load_jsonl_data(config.train_data_path)
print(f"로드된 샘플 수: {len(raw_data)}")

# 2. 모델, 토크나이저 설정
model, tokenizer = setup_model_and_tokenizer(config)

# 3. 데이터셋 준비
print("데이터셋 준비 중...")
dataset = prepare_dataset(raw_data, tokenizer, config)

# 학습/검증 분할 (9:1)
split_dataset = dataset.train_test_split(test_size=0.1, seed=config.seed)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

print(f"학습 샘플: {len(train_dataset)}, 검증 샘플: {len(eval_dataset)}")

# 4. 학습 설정
training_args = TrainingArguments(
    output_dir=config.output_dir,
    overwrite_output_dir=True,
    
    # 배치 크기
    per_device_train_batch_size=config.per_device_train_batch_size,
    per_device_eval_batch_size=config.per_device_eval_batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    
    # 에폭
    num_train_epochs=config.num_train_epochs,
    
    # 학습률
    learning_rate=config.learning_rate,
    warmup_ratio=config.warmup_ratio,
    
    # 옵티마이저
    optim="paged_adamw_32bit",  # 8-bit 최적화
    
    # 저장 및 로깅
    save_strategy="epoch",
    logging_steps=10,
    eval_strategy="epoch",
    
    # 계산 최적화
    fp16=config.fp16,
    gradient_checkpointing=True,
    max_grad_norm=1.0,
    
    # 기타
    seed=config.seed,
    report_to=["tensorboard"],
    logging_dir="./logs",
)

# 5. Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer, pad_to_multiple_of=8),
)

# 6. 학습 시작
print("학습 시작...")
trainer.train()

# 7. 모델 저장
print("모델 저장 중...")
model.save_pretrained(config.output_dir)
tokenizer.save_pretrained(config.output_dir)

print(f"✓ 파인튜닝 완료: {config.output_dir}")



데이터 로딩 중...
로드된 샘플 수: 1453
모델 로딩: Qwen/Qwen2.5-1.5B-Instruct


`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


trainable params: 1,089,536 || all params: 1,544,803,840 || trainable%: 0.0705
데이터셋 준비 중...


포매팅 중...:   0%|          | 0/1453 [00:00<?, ? examples/s]

학습 샘플: 1307, 검증 샘플: 146


...

학습 시작...


c:\Users\helen\anaconda3\envs\LLMenv\lib\site-packages\bitsandbytes\autograd\_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Epoch,Training Loss,Validation Loss
1,0.210900,0.213300
2,0.188800,0.204048
3,0.184000,0.199951
4,0.191600,0.197915
5,0.171100,0.195824
6,0.168600,0.196182
7,0.162700,0.195154
8,0.156100,0.196050
9,0.150600,0.196372
10,0.170000,0.196334


c:\Users\helen\anaconda3\envs\LLMenv\lib\site-packages\bitsandbytes\autograd\_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
c:\Users\helen\anaconda3\envs\LLMenv\lib\site-packages\bitsandbytes\autograd\_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
c:\Users\helen\anaconda3\envs\LLMenv\lib\site-packages\bitsandbytes\autograd\_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
c:\Users\helen\anaconda3\envs\LLMenv\lib\site-packages\bitsandbytes\autograd\_functions.py:181: UserWarning: MatMul8bitLt

모델 저장 중...
✓ 파인튜닝 완료: ./htp_lora_model


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import AutoPeftModelForCausalLM
import json
import os
import warnings
import time

warnings.filterwarnings('ignore')

def check_device_placement(model):
    """모델이 실제로 어느 디바이스에 있는지 확인"""
    print("\n" + "=" * 50)
    print("🔍 디바이스 배치 확인")
    print("=" * 50)
    
    devices = {}
    for name, param in model.named_parameters():
        device = str(param.device)
        devices[device] = devices.get(device, 0) + 1
    
    for device, count in devices.items():
        print(f"  {device}: {count} 파라미터")
    
    print(f"\n✓ 주 디바이스: {next(model.parameters()).device}")
    return next(model.parameters()).device

def load_finetuned_model_debug(model_path, base_model_name=None, use_4bit=False):
    """
    디버깅용 모델 로드 (4-bit 양자화 기본 OFF)
    """
    try:
        print("=" * 50)
        print("단계 1: 설정 확인")
        print("=" * 50)
        
        # CUDA 사용 가능 여부 확인
        print(f"🔧 CUDA 사용 가능: {torch.cuda.is_available()}")
        if torch.cuda.is_available():
            print(f"🔧 GPU 이름: {torch.cuda.get_device_name(0)}")
            print(f"🔧 GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
        
        # 설정 파일 로드
        adapter_config_path = os.path.join(model_path, "adapter_config.json")
        if os.path.exists(adapter_config_path):
            with open(adapter_config_path, 'r') as f:
                config = json.load(f)
                base_model_name = config.get("base_model_name_or_path", base_model_name)
        
        if base_model_name is None:
            base_model_name = "Qwen/Qwen2.5-1.5B-Instruct"
        
        print(f"\n베이스 모델: {base_model_name}")
        
        print("\n" + "=" * 50)
        print("단계 2: 모델 로드 (양자화 없이)")
        print("=" * 50)
        
        # 양자화 없이 로드 (디버깅용)
        if use_4bit:
            print("⚠️ 4-bit 양자화 사용 (느릴 수 있음)")
            quantization_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.float16,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4"
            )
        else:
            print("✓ FP16으로 로드 (빠름)")
            quantization_config = None
        
        model = AutoPeftModelForCausalLM.from_pretrained(
            model_path,
            device_map="auto",
            torch_dtype=torch.float16,
            quantization_config=quantization_config,
            trust_remote_code=True
        )
        
        tokenizer = AutoTokenizer.from_pretrained(base_model_name)
        tokenizer.pad_token = tokenizer.eos_token
        
        print("\n✓ LoRA 가중치 병합 중...")
        model = model.merge_and_unload()
        
        # 디바이스 확인
        device = check_device_placement(model)
        
        print("\n✓ 모델 로드 완료!")
        return model, tokenizer
        
    except Exception as e:
        print(f"\n❌ 모델 로드 실패: {str(e)}")
        import traceback
        traceback.print_exc()
        raise

def simple_generate(model, tokenizer, text, max_new_tokens=50):
    """
    최소한의 설정으로 생성 테스트
    """
    print("\n" + "=" * 50)
    print("🧪 간단 생성 테스트")
    print("=" * 50)
    
    # 입력 준비
    inputs = tokenizer(text, return_tensors="pt")
    input_ids = inputs['input_ids'].to(model.device)
    
    print(f"입력 텍스트: {text}")
    print(f"입력 토큰 수: {input_ids.shape[1]}")
    print(f"입력 디바이스: {input_ids.device}")
    print(f"모델 디바이스: {next(model.parameters()).device}")
    
    # 생성 시작
    print(f"\n⏱️ 생성 시작... (max_new_tokens={max_new_tokens})")
    start_time = time.time()
    
    try:
        with torch.no_grad():
            # 매우 간단한 생성 설정
            outputs = model.generate(
                input_ids,
                max_new_tokens=max_new_tokens,
                do_sample=False,  # greedy decoding (가장 빠름)
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
            )
        
        elapsed = time.time() - start_time
        print(f"✓ 생성 완료! ({elapsed:.2f}초)")
        
        # 결과 디코딩
        generated_ids = outputs[0][input_ids.shape[1]:]
        result = tokenizer.decode(generated_ids, skip_special_tokens=True)
        
        print(f"생성된 토큰 수: {len(generated_ids)}")
        print(f"초당 토큰: {len(generated_ids)/elapsed:.2f} tokens/sec")
        print(f"\n결과:\n{result}")
        
        return result
        
    except Exception as e:
        elapsed = time.time() - start_time
        print(f"❌ 생성 실패 ({elapsed:.2f}초 경과)")
        import traceback
        traceback.print_exc()
        raise

def test_tokenizer(tokenizer):
    """토크나이저 테스트"""
    print("\n" + "=" * 50)
    print("🔤 토크나이저 테스트")
    print("=" * 50)
    
    test_texts = [
        "안녕하세요",
        "나무가 크고 가지가 많으며 뿌리가 깊게 표현됨",
        "Hello world"
    ]
    
    for text in test_texts:
        tokens = tokenizer.encode(text)
        print(f"'{text}' → {len(tokens)} 토큰")
    
    print(f"\nPad token: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")
    print(f"EOS token: {tokenizer.eos_token} (ID: {tokenizer.eos_token_id})")

# 메인 실행
if __name__ == "__main__":
    try:
        print("🚀 HTP 모델 디버깅 시작\n")
        
        # 1단계: 모델 로드
        model, tokenizer = load_finetuned_model_debug(
            "./htp_lora_model",
            base_model_name="Qwen/Qwen2.5-1.5B-Instruct",
            use_4bit=False  # 디버깅시 False로!
        )
        
        # 2단계: 토크나이저 테스트
        test_tokenizer(tokenizer)
        
        # 3단계: 초간단 생성 테스트 (영어)
        print("\n" + "=" * 70)
        print("테스트 1: 영어 생성")
        print("=" * 70)
        simple_generate(model, tokenizer, "The tree is", max_new_tokens=20)
        
        # 4단계: 한국어 생성 테스트
        print("\n" + "=" * 70)
        print("테스트 2: 한국어 생성")
        print("=" * 70)
        simple_generate(model, tokenizer, "나무가", max_new_tokens=20)
        
        # 5단계: 프롬프트 형식으로 테스트
        print("\n" + "=" * 70)
        print("테스트 3: Qwen 프롬프트 형식")
        print("=" * 70)
        qwen_prompt = """<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Say hello<|im_end|>
<|im_start|>assistant
"""
        simple_generate(model, tokenizer, qwen_prompt, max_new_tokens=20)
        
        print("\n" + "=" * 70)
        print("✓ 모든 테스트 완료!")
        print("=" * 70)
        
    except Exception as e:
        print(f"\n❌ 에러 발생: {str(e)}")
        import traceback
        traceback.print_exc()

🚀 HTP 모델 디버깅 시작

단계 1: 설정 확인
🔧 CUDA 사용 가능: True
🔧 GPU 이름: NVIDIA GeForce RTX 4060
🔧 GPU 메모리: 8.00 GB

베이스 모델: Qwen/Qwen2.5-1.5B-Instruct

단계 2: 모델 로드 (양자화 없이)
✓ FP16으로 로드 (빠름)


`torch_dtype` is deprecated! Use `dtype` instead!
Exception in thread Thread-4 (_readerthread):
Traceback (most recent call last):
  File "c:\Users\helen\anaconda3\envs\LLMenv\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\helen\anaconda3\envs\LLMenv\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\helen\anaconda3\envs\LLMenv\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\helen\anaconda3\envs\LLMenv\lib\subprocess.py", line 1515, in _readerthread
    buffer.append(fh.read())
  File "c:\Users\helen\anaconda3\envs\LLMenv\lib\codecs.py", line 322, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc0 in position 6: invalid start byte
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBO


✓ LoRA 가중치 병합 중...

🔍 디바이스 배치 확인
  cuda:0: 338 파라미터

✓ 주 디바이스: cuda:0

✓ 모델 로드 완료!

🔤 토크나이저 테스트
'안녕하세요' → 3 토큰
'나무가 크고 가지가 많으며 뿌리가 깊게 표현됨' → 19 토큰
'Hello world' → 2 토큰

Pad token: <|im_end|> (ID: 151645)
EOS token: <|im_end|> (ID: 151645)

테스트 1: 영어 생성

🧪 간단 생성 테스트
입력 텍스트: The tree is
입력 토큰 수: 3
입력 디바이스: cuda:0
모델 디바이스: cuda:0

⏱️ 생성 시작... (max_new_tokens=20)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


✓ 생성 완료! (1.89초)
생성된 토큰 수: 20
초당 토큰: 10.58 tokens/sec

결과:
 very tall, and the branches are lush. The leaves are green and shiny, with a strong sense

테스트 2: 한국어 생성

🧪 간단 생성 테스트
입력 텍스트: 나무가
입력 토큰 수: 3
입력 디바이스: cuda:0
모델 디바이스: cuda:0

⏱️ 생성 시작... (max_new_tokens=20)
✓ 생성 완료! (1.32초)
생성된 토큰 수: 20
초당 토큰: 15.11 tokens/sec

결과:
무한무한정수

Given the function \( f(x) = \frac{1}{

테스트 3: Qwen 프롬프트 형식

🧪 간단 생성 테스트
입력 텍스트: <|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Say hello<|im_end|>
<|im_start|>assistant

입력 토큰 수: 21
입력 디바이스: cuda:0
모델 디바이스: cuda:0

⏱️ 생성 시작... (max_new_tokens=20)
✓ 생성 완료! (0.71초)
생성된 토큰 수: 10
초당 토큰: 14.13 tokens/sec

결과:
Hello! How can I assist you today?

✓ 모든 테스트 완료!


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import AutoPeftModelForCausalLM
import json
import os
import time

def load_htp_model(model_path="./htp_lora_model", base_model_name=None):
    """
    HTP LoRA 모델 로드
    """
    print("=" * 50)
    print("🚀 HTP 모델 로드 중...")
    print("=" * 50)
    
    # CUDA 확인
    if torch.cuda.is_available():
        print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
        print(f"✓ GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB\n")
    else:
        print("⚠️ CPU 모드 (매우 느릴 수 있음)\n")
    
    # 베이스 모델 자동 탐지
    if base_model_name is None:
        config_path = os.path.join(model_path, "adapter_config.json")
        if os.path.exists(config_path):
            with open(config_path, 'r') as f:
                config = json.load(f)
                base_model_name = config.get("base_model_name_or_path", "Qwen/Qwen2.5-1.5B-Instruct")
        else:
            base_model_name = "Qwen/Qwen2.5-1.5B-Instruct"
    
    print(f"베이스 모델: {base_model_name}")
    
    try:
        # 모델 로드
        model = AutoPeftModelForCausalLM.from_pretrained(
            model_path,
            device_map="auto",
            dtype=torch.float16,  # torch_dtype 대신 dtype 사용
            trust_remote_code=True
        )
        
        # LoRA 병합
        print("LoRA 가중치 병합 중...")
        model = model.merge_and_unload()
        
        # 토크나이저 로드
        tokenizer = AutoTokenizer.from_pretrained(base_model_name)
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
        
        print("✓ 모델 로드 완료!\n")
        return model, tokenizer
        
    except Exception as e:
        print(f"❌ 에러: {str(e)}")
        raise

def format_htp_prompt(htp_description, lang="ko"):
    """
    HTP 해석용 프롬프트 생성
    
    Args:
        htp_description: HTP 그림 특징 설명
        lang: 'ko' (한국어) 또는 'en' (영어)
    """
    if lang == "ko":
        system_msg = """당신은 HTP(집-나무-사람) 그림검사 해석 전문 심리학자입니다.
주어진 그림 특징을 바탕으로 심리학적 해석을 제공하세요.
해석은 명확하고 전문적이어야 하며, 가능한 심리적 의미를 설명해주세요."""
        
        prompt = f"""<|im_start|>system
{system_msg}<|im_end|>
<|im_start|>user
다음 HTP 그림 특징을 해석해주세요:
{htp_description}<|im_end|>
<|im_start|>assistant
"""
    else:
        system_msg = """You are a professional psychologist specialized in HTP (House-Tree-Person) test interpretation.
Provide psychological interpretation based on the given drawing features.
Your interpretation should be clear, professional, and explain possible psychological meanings."""
        
        prompt = f"""<|im_start|>system
{system_msg}<|im_end|>
<|im_start|>user
Please interpret the following HTP drawing features:
{htp_description}<|im_end|>
<|im_start|>assistant
"""
    
    return prompt

def generate_htp_interpretation(
    model,
    tokenizer,
    htp_description,
    max_new_tokens=256,
    temperature=0.7,
    top_p=0.9,
    lang="ko",
    verbose=True
):
    """
    HTP 해석 생성
    
    Args:
        model: 로드된 모델
        tokenizer: 토크나이저
        htp_description: HTP 그림 특징 설명
        max_new_tokens: 생성할 최대 토큰 수
        temperature: 생성 다양성 (높을수록 창의적)
        top_p: nucleus sampling
        lang: 'ko' 또는 'en'
        verbose: 진행 상황 출력 여부
    
    Returns:
        str: 생성된 해석
    """
    # 프롬프트 생성
    prompt = format_htp_prompt(htp_description, lang=lang)
    
    if verbose:
        print("=" * 70)
        print("📝 HTP 해석 생성 중...")
        print("=" * 70)
        print(f"입력: {htp_description}\n")
    
    # 토큰화
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    )
    
    # GPU로 이동
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    input_length = inputs['input_ids'].shape[1]
    
    if verbose:
        print(f"입력 토큰 수: {input_length}")
        print(f"생성 시작... (최대 {max_new_tokens} 토큰)")
    
    # 생성
    start_time = time.time()
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.2,
            num_return_sequences=1
        )
    
    elapsed = time.time() - start_time
    
    # 디코딩 (입력 제외)
    generated_tokens = outputs[0][input_length:]
    interpretation = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    
    if verbose:
        print(f"✓ 생성 완료! ({elapsed:.2f}초)")
        print(f"생성된 토큰 수: {len(generated_tokens)}")
        print(f"속도: {len(generated_tokens)/elapsed:.1f} tokens/sec")
        print("=" * 70)
    
    return interpretation.strip()

def batch_interpret(model, tokenizer, htp_list, **kwargs):
    """
    여러 HTP 설명을 한번에 해석
    
    Args:
        model: 모델
        tokenizer: 토크나이저
        htp_list: HTP 설명 리스트
        **kwargs: generate_htp_interpretation에 전달할 추가 인자
    
    Returns:
        list: 해석 결과 리스트
    """
    results = []
    
    for i, htp_desc in enumerate(htp_list, 1):
        print(f"\n{'='*70}")
        print(f"[{i}/{len(htp_list)}] 해석 중...")
        print(f"{'='*70}")
        
        interpretation = generate_htp_interpretation(
            model, tokenizer, htp_desc, **kwargs
        )
        
        results.append({
            'input': htp_desc,
            'interpretation': interpretation
        })
        
        print(f"\n입력: {htp_desc}")
        print(f"해석: {interpretation}")
    
    return results

# 사용 예시
if __name__ == "__main__":
    try:
        # 1. 모델 로드
        model, tokenizer = load_htp_model("./htp_lora_model")
        
        
        # 3. 배치 해석 테스트
        print("\n" + "="*70)
        print("배치 해석 테스트")
        print("="*70)
        
        test_cases = [
            "The tree is large with many branches and deeply drawn roots",
            "The house is small with no doors or windows, and no smoke from the chimney",
            "The person's hands are hidden behind their back and the face has no expression",
            "The tree trunk has holes and the branches are drooping downward",
            "The house has an excessively large roof with many complex decorations",
            "The tree is drawn in the corner with very thin trunk and sparse leaves",
            "The person is drawn very small at the bottom of the page with no facial features",
            "The house has heavily shaded walls and windows with bars",
            "The tree has broken branches and appears to be leaning to one side",
            "The person has unusually large hands and feet with exaggerated details"
        ]
        
        results = batch_interpret(
            model,
            tokenizer,
            test_cases,
            max_new_tokens=150,
            temperature=0.7,
            lang="en",  # 영어로 변경
            verbose=False
        )
        
        # 4. 결과 저장 (선택사항)
        print("\n" + "="*70)
        print("결과 저장")
        print("="*70)
        
        output_file = "htp_interpretations.json"
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
        
        print(f"✓ 결과 저장됨: {output_file}")
        
    except Exception as e:
        print(f"\n❌ 에러 발생: {str(e)}")
        import traceback
        traceback.print_exc()

🚀 HTP 모델 로드 중...
✓ GPU: NVIDIA GeForce RTX 4060
✓ GPU 메모리: 8.0 GB

베이스 모델: Qwen/Qwen2.5-1.5B-Instruct


Exception in thread Thread-4 (_readerthread):
Traceback (most recent call last):
  File "c:\Users\helen\anaconda3\envs\LLMenv\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\helen\anaconda3\envs\LLMenv\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\helen\anaconda3\envs\LLMenv\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\helen\anaconda3\envs\LLMenv\lib\subprocess.py", line 1515, in _readerthread
    buffer.append(fh.read())
  File "c:\Users\helen\anaconda3\envs\LLMenv\lib\codecs.py", line 322, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc0 in position 6: invalid start byte


LoRA 가중치 병합 중...
✓ 모델 로드 완료!


배치 해석 테스트

[1/10] 해석 중...

입력: The tree is large with many branches and deeply drawn roots
해석: This indicates significant growth potential and potentially underlying issues related to dependency or difficulty establishing boundaries. The expansive nature of both the tree and its roots suggests an overwhelming need for external support and resources, possibly reflecting feelings of insecurity or lack of self-esteem. It could also signify difficulties in forming healthy relationships due to excessive reliance on others.

[2/10] 해석 중...

입력: The house is small with no doors or windows, and no smoke from the chimney
해석: This suggests an underlying feeling of isolation, detachment, and possibly unresolved grief or loss. The lack of visual stimuli within the structure could indicate difficulty forming relationships or experiencing feelings of emptiness. It may also reflect difficulties coping with external pressures or challenges.

[3/10] 해석 중...

입력: The perso

In [ ]:
from huggingface_hub import HfApi
import os

# ✅ 올바른 방법 1: 토큰을 직접 입력
token = "your_huggingface_token_here"  # 여기에 실제 토큰 입력

# 또는
# ✅ 올바른 방법 2: 환경 변수에서 가져오기 (환경 변수 이름은 HF_TOKEN 같은 것)
# token = os.getenv("HF_TOKEN")  # 시스템 환경 변수에 HF_TOKEN=your_token 설정 필요

api = HfApi(token=token)

# 먼저 레포가 있는지 확인하고 없으면 생성
try:
    api.repo_info(repo_id="helena29/Qwen2.5_LoRA_for_HTP", repo_type="model")
    print("✓ 레포지토리가 이미 존재합니다.")
except:
    print("레포지토리가 없습니다. 생성 중...")
    api.create_repo(
        repo_id="helena29/Qwen2.5_LoRA_for_HTP",
        repo_type="model",
        private=False  # True로 설정하면 비공개
    )
    print("✓ 레포지토리 생성 완료!")

# LoRA 모델 폴더만 업로드 (전체 LoRa 폴더가 아닌 모델 폴더만)
print("\n업로드 시작...")
api.upload_folder(
    folder_path=r"C:\Users\helen\Desktop\kt cloud tech up\basic_project\models\LoRa\htp_lora_model",  # ✅ 수정: 모델 폴더만 업로드
    repo_id="helena29/Qwen2.5_LoRA_for_HTP",
    repo_type="model",
)

print("✅ 업로드 완료!")

✓ 레포지토리가 이미 존재합니다.

업로드 시작...


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

✅ 업로드 완료!


# 🔄 전체 모델 업로드 (Base Model + LoRA 병합)

현재 HuggingFace에는 LoRA 어댑터만 업로드된 상태입니다.
아래 코드는 Base Model과 LoRA를 병합한 전체 모델을 업로드합니다.

**주의사항:**
- 전체 모델은 약 3GB 정도의 크기입니다
- 업로드 시간이 어댑터만 올릴 때보다 오래 걸립니다
- 사용자는 이 모델을 바로 사용할 수 있습니다 (LoRA 로딩 불필요)

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import AutoPeftModelForCausalLM
from huggingface_hub import HfApi
import os

# ========== 설정 ==========
LORA_MODEL_PATH = "./htp_lora_model"  # 로컬 LoRA 모델 경로
BASE_MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"  # Base 모델
MERGED_MODEL_PATH = "./htp_merged_full_model"  # 병합된 모델 저장 경로
HF_REPO_NAME = "helena29/Qwen2.5_LoRA_for_HTP_Full"  # HuggingFace 레포 이름 (새로운 레포)

# HuggingFace 토큰 (여기에 실제 토큰 입력)
HF_TOKEN = "your_huggingface_token_here"  # ⚠️ 실제 토큰으로 변경 필요!

print("=" * 70)
print("🔄 전체 모델 생성 및 업로드")
print("=" * 70)

# ========== 1단계: LoRA 모델 로드 및 병합 ==========
print("\n[1/4] LoRA 모델 로드 및 Base Model과 병합 중...")
print("-" * 70)

try:
    # LoRA 모델 로드
    print(f"LoRA 모델 경로: {LORA_MODEL_PATH}")
    model = AutoPeftModelForCausalLM.from_pretrained(
        LORA_MODEL_PATH,
        device_map="auto",
        torch_dtype=torch.float16,
        trust_remote_code=True
    )
    
    print("✓ LoRA 모델 로드 완료")
    
    # LoRA를 Base Model에 병합
    print("LoRA 가중치를 Base Model에 병합 중...")
    merged_model = model.merge_and_unload()
    print("✓ 병합 완료!")
    
    # 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    print("✓ 토크나이저 로드 완료")
    
except Exception as e:
    print(f"❌ 에러: {str(e)}")
    raise

# ========== 2단계: 병합된 모델 로컬 저장 ==========
print(f"\n[2/4] 병합된 전체 모델을 로컬에 저장 중...")
print(f"저장 경로: {MERGED_MODEL_PATH}")
print("-" * 70)

try:
    os.makedirs(MERGED_MODEL_PATH, exist_ok=True)
    
    # 모델 저장
    merged_model.save_pretrained(
        MERGED_MODEL_PATH,
        safe_serialization=True,  # safetensors 형식으로 저장
        max_shard_size="2GB"  # 샤드 크기 제한
    )
    
    # 토크나이저 저장
    tokenizer.save_pretrained(MERGED_MODEL_PATH)
    
    print("✓ 로컬 저장 완료!")
    
    # 저장된 파일 확인
    saved_files = os.listdir(MERGED_MODEL_PATH)
    print(f"저장된 파일 수: {len(saved_files)}")
    
    # 모델 파일 크기 확인
    total_size = 0
    for file in saved_files:
        file_path = os.path.join(MERGED_MODEL_PATH, file)
        if os.path.isfile(file_path):
            size = os.path.getsize(file_path) / (1024**3)  # GB
            total_size += size
            if size > 0.01:  # 10MB 이상인 파일만 출력
                print(f"  - {file}: {size:.2f} GB")
    
    print(f"전체 크기: {total_size:.2f} GB")
    
except Exception as e:
    print(f"❌ 저장 실패: {str(e)}")
    raise


🔄 전체 모델 생성 및 업로드

[1/4] LoRA 모델 로드 및 Base Model과 병합 중...
----------------------------------------------------------------------
LoRA 모델 경로: ./htp_lora_model


`torch_dtype` is deprecated! Use `dtype` instead!
Exception in thread Thread-4 (_readerthread):
Traceback (most recent call last):
  File "c:\Users\helen\anaconda3\envs\LLMenv\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\helen\anaconda3\envs\LLMenv\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\helen\anaconda3\envs\LLMenv\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\helen\anaconda3\envs\LLMenv\lib\subprocess.py", line 1515, in _readerthread
    buffer.append(fh.read())
  File "c:\Users\helen\anaconda3\envs\LLMenv\lib\codecs.py", line 322, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc0 in position 6: invalid start byte
Exception in thread Thread-4 (_readerthread):
Traceback (most recent call last):
  File "c:\Users\helen\anaconda3\envs\LLMe

✓ LoRA 모델 로드 완료
LoRA 가중치를 Base Model에 병합 중...
✓ 병합 완료!
✓ 토크나이저 로드 완료

[2/4] 병합된 전체 모델을 로컬에 저장 중...
저장 경로: ./htp_merged_full_model
----------------------------------------------------------------------
✓ 토크나이저 로드 완료

[2/4] 병합된 전체 모델을 로컬에 저장 중...
저장 경로: ./htp_merged_full_model
----------------------------------------------------------------------
✓ 로컬 저장 완료!
저장된 파일 수: 12
  - model-00001-of-00002.safetensors: 1.84 GB
✓ 로컬 저장 완료!
저장된 파일 수: 12
  - model-00001-of-00002.safetensors: 1.84 GB
  - model-00002-of-00002.safetensors: 1.04 GB
  - tokenizer.json: 0.01 GB
전체 크기: 2.89 GB
  - model-00002-of-00002.safetensors: 1.04 GB
  - tokenizer.json: 0.01 GB
전체 크기: 2.89 GB


## 📌 추가 옵션: 기존 레포에 덮어쓰기

만약 새 레포가 아닌 기존 레포(`helena29/Qwen2.5_LoRA_for_HTP`)에 전체 모델을 덮어쓰고 싶다면 아래 코드를 사용하세요.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import AutoPeftModelForCausalLM
from huggingface_hub import HfApi
import os

# ========== 설정 ==========
LORA_MODEL_PATH = "./htp_lora_model"  # 로컬 LoRA 모델 경로
BASE_MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"  # Base 모델
MERGED_MODEL_PATH = "./htp_merged_full_model"  # 병합된 모델 저장 경로
HF_REPO_NAME = "helena29/Qwen2.5_LoRA_for_HTP_Full"  # HuggingFace 레포 이름 (새로운 레포)

# HuggingFace 토큰 (환경 변수에서 가져오기)
HF_TOKEN = os.getenv("HF_TOKEN")  # 환경 변수에서 토큰 가져오기
if not HF_TOKEN:
    raise ValueError("HF_TOKEN 환경 변수가 설정되지 않았습니다. 먼저 설정해주세요.")

print("=" * 70)
print("🔄 전체 모델 생성 및 업로드")
print("=" * 70)

# ========== 1단계: LoRA 모델 로드 및 병합 ==========
print("\n[1/4] LoRA 모델 로드 및 Base Model과 병합 중...")
print("-" * 70)

try:
    # LoRA 모델 로드
    print(f"LoRA 모델 경로: {LORA_MODEL_PATH}")
    model = AutoPeftModelForCausalLM.from_pretrained(
        LORA_MODEL_PATH,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
    )
    
    print(f"✅ LoRA 모델 로드 완료!")
    print(f"Base 모델: {BASE_MODEL_NAME}")
    
    # 병합
    print("\n병합 시작...")
    merged_model = model.merge_and_unload()
    print("✅ 병합 완료!")
    
except Exception as e:
    print(f"❌ 오류 발생: {e}")
    raise


⚠️ 기존 레포에 전체 모델 덮어쓰기 모드
[1/3] LoRA 모델 병합 중...
✓ 병합 완료

[2/3] 로컬 저장 중: ./htp_merged_full_model
✓ 병합 완료

[2/3] 로컬 저장 중: ./htp_merged_full_model
✓ 저장 완료

[3/3] 기존 레포에 업로드 중: helena29/Qwen2.5_LoRA_for_HTP
⚠️ 기존 파일들을 삭제하고 새 파일로 교체합니다...
✓ 저장 완료

[3/3] 기존 레포에 업로드 중: helena29/Qwen2.5_LoRA_for_HTP
⚠️ 기존 파일들을 삭제하고 새 파일로 교체합니다...


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

✓ 업로드 완료!
✓ URL: https://huggingface.co/helena29/Qwen2.5_LoRA_for_HTP


## 🔧 문제 해결: adapter_config.json 파일 업로드

HuggingFace에서 다운로드 시 `adapter_config.json` 파일이 없다는 오류가 발생하면 아래 코드로 누락된 파일들을 다시 업로드하세요.